In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.metrics import roc_auc_score

In [2]:
# 読み出し
data = pd.read_pickle('data.pkl')
nomination_onehot = pd.read_pickle('nomination_onehot.pkl')
selected_performers_onehot = pd.read_pickle('selected_performers_onehot.pkl')
selected_directors_onehot = pd.read_pickle('selected_directors_onehot.pkl')
selected_studio_onehot = pd.read_pickle('selected_studio_onehot.pkl')
selected_scriptwriter_onehot = pd.read_pickle('selected_scriptwriter_onehot.pkl')
review_dataframe = pd.read_pickle('review_dataframe.pkl')
tfidf = pd.read_pickle('tfidf02_035.pkl')

In [3]:
from sklearn.decomposition import PCA

pca = PCA(n_components=20)
pca.fit(tfidf.values)
tfidf_df = pd.DataFrame(pca.transform(tfidf.values), index = tfidf.index)

In [4]:
table = pd.concat([
    data[['prize', 'title', 'year', 'screen_time']],
    nomination_onehot,
    selected_performers_onehot,
    selected_directors_onehot,
    selected_studio_onehot,
    selected_scriptwriter_onehot,
    tfidf_df ### pca
], axis = 1)

review_len = review_dataframe['reviews'].map(lambda s: len(s))
review_len.name = 'len'
year = table['year']

temp = pd.concat([year, review_len], axis = 1)

group = temp.groupby('year')

temp = temp.apply( lambda s: (s['len']-group.mean()['len'][s['year']] )/group.std()['len'][s['year']] , axis = 1)
temp.name = 'len'

table['len'] = temp


In [135]:
# invalid

def logregobj(preds, dtrain):
    grad = - preds + dtrain
    grad[dtrain == 1] = 1
    grad[dtrain == 0] = -1
    hess = np.zeros(len(preds))
    return grad, hess


In [5]:
import numpy as np
import optuna

In [ ]:
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth',1,10),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.05, 0.2),
        'n_estimators': trial.suggest_int('n_estimators ', 20, 200),
        'n_jobs': 4,
        'gamma': trial.suggest_uniform('gamma', 0, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0.5, 3),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 3)
    }

    print(param)
    
    l = []

    for year in range(1978, 2019 + 1):
        rg = xgb.XGBRegressor(silent= True, **param)
        X = table.query('year != {}'.format(year)).drop(['prize', 'title'], axis = 1).values
        y = table.query('year != {}'.format(year))['prize'].values
        rg.fit(X,y)
        result = rg.predict(table.query('year == {}'.format(year)).drop(['prize', 'title'], axis = 1).values)
        scope = table.query('year == {}'.format(year))[['title','prize']]
        scope['result'] = result
        l.append(scope)
        title = scope['title'].copy()
        title[scope['prize'] == 1] = title[scope['prize'] == 1].map(lambda s: '★' + s)
        #print(year)
        #print(pd.Series(result, index = title.values).sort_values(ascending=False) )
        #print('')
    

    temp = pd.concat(l)

    print(roc_auc_score(temp['prize'], temp['result']))
    return -roc_auc_score(temp['prize'], temp['result'])

study = optuna.create_study()
study.optimize(objective, n_trials=200)

{'max_depth': 10, 'learning_rate': 0.09593066665668842, 'n_estimators': 50, 'n_jobs': 4, 'gamma': 0.7210236947460333, 'reg_lambda': 2.6062041118604995, 'reg_alpha': 0.03522344465232996}
0.7106227106227105


[I 2019-08-21 09:18:50,011] Finished trial#0 resulted in value: -0.7106227106227105. Current best value is -0.7106227106227105 with parameters: {'max_depth': 10, 'learning_rate': 0.09593066665668842, 'n_estimators ': 50, 'gamma': 0.7210236947460333, 'reg_lambda': 2.6062041118604995, 'reg_alpha': 0.03522344465232996}.


{'max_depth': 9, 'learning_rate': 0.19771299339430942, 'n_estimators': 147, 'n_jobs': 4, 'gamma': 0.784551572958621, 'reg_lambda': 1.485711552802786, 'reg_alpha': 1.2529383657778181}
0.69392786700479


[I 2019-08-21 09:19:12,518] Finished trial#1 resulted in value: -0.69392786700479. Current best value is -0.7106227106227105 with parameters: {'max_depth': 10, 'learning_rate': 0.09593066665668842, 'n_estimators ': 50, 'gamma': 0.7210236947460333, 'reg_lambda': 2.6062041118604995, 'reg_alpha': 0.03522344465232996}.


{'max_depth': 6, 'learning_rate': 0.12218873206333361, 'n_estimators': 185, 'n_jobs': 4, 'gamma': 0.8476605356413363, 'reg_lambda': 0.6209083111081797, 'reg_alpha': 0.0179961577097697}
0.6780783319244857


...

{'max_depth': 1, 'learning_rate': 0.07751131137563419, 'n_estimators': 87, 'n_jobs': 4, 'gamma': 0.14944859099059687, 'reg_lambda': 1.885138937224624, 'reg_alpha': 0.5191575378670711}
0.7745843899690052


[I 2019-08-21 09:22:11,877] Finished trial#16 resulted in value: -0.7745843899690052. Current best value is -0.7907156945618483 with parameters: {'max_depth': 1, 'learning_rate': 0.06309331582801181, 'n_estimators ': 81, 'gamma': 0.5839936400995296, 'reg_lambda': 2.01330814334193, 'reg_alpha': 0.7699132857643405}.


{'max_depth': 3, 'learning_rate': 0.10342074777466811, 'n_estimators': 33, 'n_jobs': 4, 'gamma': 0.5061565991188393, 'reg_lambda': 2.886174765196138, 'reg_alpha': 0.8737572216945211}
0.7395040856579318


[I 2019-08-21 09:22:15,390] Finished trial#17 resulted in value: -0.7395040856579318. Current best value is -0.7907156945618483 with parameters: {'max_depth': 1, 'learning_rate': 0.06309331582801181, 'n_estimators ': 81, 'gamma': 0.5839936400995296, 'reg_lambda': 2.01330814334193, 'reg_alpha': 0.7699132857643405}.


{'max_depth': 5, 'learning_rate': 0.06762323245351581, 'n_estimators': 119, 'n_jobs': 4, 'gamma': 0.641331300417668, 'reg_lambda': 1.3335297332536808, 'reg_alpha': 1.5897095108093091}


In [151]:

best ={'max_depth': 9,
 'learning_rate': 0.09086768010850159,
 'n_estimators ': 76,
 'gamma': 0.29703555171643825,
 'reg_lambda': 1.258947041854579,
 'reg_alpha': 0.34103840974717364}

best = study.best_params

In [234]:
l = []

allelement = 0

count = 0

for year in range(1978, 2019 + 1):
    param = {
        'silent': True,
        'max_depth': 10,
    }
    rg = xgb.XGBRegressor(silent = True,**best)
    X = table.query('year != {}'.format(year)).drop(['prize', 'title'], axis = 1).values
    y = table.query('year != {}'.format(year))['prize'].values
    rg.fit(X,y)
    result = rg.predict(table.query('year == {}'.format(year)).drop(['prize', 'title'], axis = 1).values)
    scope = table.query('year == {}'.format(year))[['title','prize']]
    scope['result'] = result
    l.append(scope)
    title = scope['title'].copy()
    title[scope['prize'] == 1] = title[scope['prize'] == 1].map(lambda s: '★' + s)
    print(year)
    disp = pd.Series(result, index = title.values).sort_values(ascending=False)
    print(disp)
    print('')
    allelement  += 1
    if '★' in disp.index[0]:
        count += 1
        
    
temp = pd.concat(l)
print(roc_auc_score(temp['prize'], temp['result']))
print(count,'/',allelement)

1978
★幸福の黄色いハンカチ       0.532143
八甲田山              0.365765
はなれ瞽女おりん          0.111901
竹山ひとり旅            0.051013
青春の門 自立篇（1977）    0.051013
dtype: float32

1979
★事件        0.441585
柳生一族の陰謀    0.287557
鬼畜         0.270099
サード        0.083551
愛の亡霊       0.052906
dtype: float32

1980
太陽を盗んだ男       0.655968
あゝ野麦峠         0.619628
衝動殺人 息子よ      0.583455
★復讐するは我にあり    0.125372
もう頬づえはつかない    0.060124
dtype: float32

1981
遙かなる山の呼び声      0.127930
二百三高地          0.097594
★ツィゴイネルワイゼン    0.051663
動乱             0.041573
父よ母よ!          0.041573
dtype: float32

1982
泥の河                0.281908
遠雷                 0.240865
ええじゃないか            0.142572
日本の熱い日々 謀殺・下山事件    0.116348
★駅/STATION         0.085043
dtype: float32

1983
★蒲田行進曲      0.614684
鬼龍院花子の生涯    0.268238
未完の対局       0.248190
誘拐報道        0.083835
疑惑          0.057400
dtype: float32

1984
戦場のメリークリスマス    0.159397
南極物語（1983）     0.085187
家族ゲーム          0.071927
細雪（1983）       0.053071
★楢山節考          0.053071
dtype: float32

1985
★お葬式        0